In [1]:
import pandas as pd
import numpy as np
from sklearn import pipeline
from sklearn.linear_model import SGDRegressor, LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
from sklearn.metrics import mean_squared_log_error, mean_squared_error
from sklearn.ensemble import RandomForestClassifier, GradientBoostingRegressor,ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.preprocessing import OneHotEncoder,LabelEncoder, Binarizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from scipy.stats import zscore
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import LocalOutlierFactor
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from xgboost import XGBClassifier, XGBRFClassifier
from sklearn.feature_selection import SelectKBest, chi2, f_classif

In [2]:
X = pd.read_csv('Training/X_train.csv')
y = pd.read_csv('Training/y_train.csv')

In [3]:
X['C6'] = X['C6'].apply(lambda x: 0 if x==False else 1)
X['C8'] = X['C8'].apply(lambda x: 0 if x==False else 1)

In [4]:
X = X.iloc[:,1:]
y = y['Dependent_Variable']
X_numerical = X.iloc[:,8:]
numerical_columns= list(X_numerical.columns)
X_categorical = X.iloc[:,:8]
categorical_columns= list(X_categorical.columns)

### Try Forward feature selection

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)

In [6]:
num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
#           ('poly', PolynomialFeatures(degree=1)),
#         ('std', StandardScaler()),
    ])

cat_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="most_frequent")),
          #('onehot', OneHotEncoder(handle_unknown='ignore')),
    ])

column_combine = ColumnTransformer([
        ("num", num_pipeline, numerical_columns),
        ("cat", cat_pipeline, categorical_columns),
    ])

full_pipeline = Pipeline(steps=[('data_pre_process', column_combine)])

In [7]:
import warnings; warnings.simplefilter('ignore')
import warnings;
warnings.filterwarnings('ignore');
from sklearn.feature_selection import RFE
from sklearn.svm import SVR


##### Select using Logistic regression

In [8]:
dictionary = {}
for i in range(10,44):
    X_train_trans = full_pipeline.fit_transform(X_train,y_train)
    X_test_trans = full_pipeline.transform(X_test)

    estimator = LogisticRegression()
    selector = RFE(estimator, n_features_to_select=i, step=1)
    selector = selector.fit(X_train_trans, y_train)

    X_train_transformed = selector.transform(X_train_trans)
    X_test_transformed = selector.transform(X_test_trans)

    rf = XGBClassifier(random_state=42)
    rf.fit(X_train_transformed,y_train)

    y_train_proba = rf.predict_proba(X_train_transformed)
    y_test_proba = rf.predict_proba(X_test_transformed)
    fpr, tpr, thresholds = roc_curve(y_train, y_train_proba[:,1], pos_label=1)
    train_auc =  auc(fpr, tpr)
    fpr, tpr, thresholds = roc_curve(y_test, y_test_proba[:,1], pos_label=1)
    test_auc =  auc(fpr, tpr)
    dictionary[i] =[train_auc,test_auc]
    print("Train AUC is: ",train_auc)
    print("Test AUC is: ",test_auc)

[12:04:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Train AUC is:  0.7314300643495465
Test AUC is:  0.6895228657046164
[12:04:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Train AUC is:  0.7394880551948808
Test AUC is:  0.6986242068917016
[12:04:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explic

Train AUC is:  0.9097251270363411
Test AUC is:  0.7649127516548282
[12:07:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Train AUC is:  0.9177771129124843
Test AUC is:  0.7707194450686202
[12:07:46] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Train AUC is:  0.9167576423029428
Test AUC is:  0.7640126519963429
[12:07:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the object

In [9]:
dictionary

{10: [0.7314300643495465, 0.6895228657046164],
 11: [0.7394880551948808, 0.6986242068917016],
 12: [0.7477396030365869, 0.705701717522226],
 13: [0.7621592524184033, 0.707976838407454],
 14: [0.8138648781319568, 0.7297207418296336],
 15: [0.8251700772491098, 0.7321555993145692],
 16: [0.8397344628041405, 0.730578173591016],
 17: [0.8507730550572006, 0.7324058175855201],
 18: [0.8578503322325989, 0.7311090565720568],
 19: [0.8579830452289618, 0.7307649260451705],
 20: [0.8627046693666081, 0.7384031228612449],
 21: [0.873839635020726, 0.7440936052186055],
 22: [0.8772748030645464, 0.7472100769994734],
 23: [0.8781728425003548, 0.7461433795718972],
 24: [0.8810663109578508, 0.7433225813091454],
 25: [0.8908929664547804, 0.7414955805492709],
 26: [0.8962516097977372, 0.744156213389229],
 27: [0.8982035848712903, 0.7483121523316827],
 28: [0.9021826924502605, 0.7621146810156588],
 29: [0.9080593358737072, 0.761601379781163],
 30: [0.9124899572979968, 0.7619225682729236],
 31: [0.90972512703

**`The above data suggests using all features.`**

##### Select using XGBClassifier

In [10]:
dictionary = {}
for i in range(10,44):
    X_train_trans = full_pipeline.fit_transform(X_train,y_train)
    X_test_trans = full_pipeline.transform(X_test)

    estimator = XGBClassifier()
    selector = RFE(estimator, n_features_to_select=i, step=1)
    selector = selector.fit(X_train_trans, y_train)

    X_train_transformed = selector.transform(X_train_trans)
    X_test_transformed = selector.transform(X_test_trans)

    rf = XGBClassifier(random_state=42)
    rf.fit(X_train_transformed,y_train)

    y_train_proba = rf.predict_proba(X_train_transformed)
    y_test_proba = rf.predict_proba(X_test_transformed)
    fpr, tpr, thresholds = roc_curve(y_train, y_train_proba[:,1], pos_label=1)
    train_auc =  auc(fpr, tpr)
    fpr, tpr, thresholds = roc_curve(y_test, y_test_proba[:,1], pos_label=1)
    test_auc =  auc(fpr, tpr)
    dictionary[i] =[train_auc,test_auc]
    print("Train AUC is: ",train_auc)
    print("Test AUC is: ",test_auc)

[12:11:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:11:17] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:11:19] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:11:20] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

[12:11:55] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:11:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:11:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:11:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

[12:12:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:12:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:12:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:12:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

[12:13:06] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:13:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:13:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:13:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

[12:13:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:13:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:13:43] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:13:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

...

Train AUC is:  0.9238830413103754
Test AUC is:  0.7688969469511535
[12:23:19] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:23:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:23:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12

[12:24:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:24:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:24:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Train AUC is:  0.9261639711380211
Test AUC is:  0.7666953692252538
[12

[12:24:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:24:43] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Train AUC is:  0.9254254261198156
Test AUC is:  0.7684020851093757
[12:24:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12

[12:25:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:25:25] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:25:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:25:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

[12:26:05] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Train AUC is:  0.928643665975442
Test AUC is:  0.7702402352694981
[12:26:06] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:26:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:

In [11]:
dictionary

{10: [0.848465955425548, 0.7489174361182248],
 11: [0.8642943525753586, 0.7508705108655194],
 12: [0.8798451787616842, 0.7542050391858537],
 13: [0.8767274903676905, 0.756220936515315],
 14: [0.8842720881194809, 0.7606279513748925],
 15: [0.8888361718374358, 0.7612939136281693],
 16: [0.8937213069439472, 0.7618196507321725],
 17: [0.8960496146017126, 0.7649007446084073],
 18: [0.9047641964037967, 0.7684475403565408],
 19: [0.9059773193266809, 0.7657819760510883],
 20: [0.9120470908201285, 0.762452164784705],
 21: [0.9099138202194832, 0.7674338025801428],
 22: [0.9139124518914065, 0.7649976586259478],
 23: [0.9117239556966718, 0.7661507639054462],
 24: [0.9171889621888166, 0.7664792423896767],
 25: [0.9184577343367738, 0.7668699002214441],
 26: [0.9188282234611076, 0.7692689509786599],
 27: [0.9200339010694565, 0.7690899173400618],
 28: [0.9269037896968709, 0.7681263518647801],
 29: [0.9238830413103754, 0.7688969469511535],
 30: [0.9282657950812208, 0.7672082416366631],
 31: [0.92616397

**`The above data also suggests using all features.`**